In [ ]:
# llama-index openai transformers accelerate optimum InstructorEmbedding sentence_transformers pypdf chromadb

In [ ]:
# !pip install -q llama-index chromadb --quiet
# !pip install -q chromadb
# !pip install -q sentence-transformers
# !pip install -q pydantic==1.10.11

In [ ]:
# import os
# os.environ["OPENAI_API_KEY"] = "YOUR-OPENAI-API-KEY"

In [ ]:
# !curl https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf --output IPCC_AR6_WGII_Chapter03.pdf

In [13]:
# import
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.llms import OpenAI
from IPython.display import Markdown, display
import chromadb


In [3]:
# define embedding function
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
# print(embed_model("This is a test sentence."))

In [4]:
documents = SimpleDirectoryReader(
    input_dir='/Users/matansharon/python/chat_with_docs/data/temp/'
).load_data()

In [5]:
print(documents)

[Document(id_='a81443a5-482b-4df5-a071-e77e954a7e7e', embedding=None, metadata={'page_label': '1', 'file_name': '2020-ml-roadmap-keynote.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='fa91b8e43cf28f6f84cf4c2347e5869b5bc1278eefb465bf26f530ca22d33e58', text='Machine Learning Roadmap 2020(a machine-learning-flavoured-visual-iterative-living-mind-map/compass)', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='6f9f1940-22ab-442f-8ecd-fb13f0a2ea73', embedding=None, metadata={'page_label': '2', 'file_name': '2020-ml-roadmap-keynote.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='913af5dfbda6f3ed39f42f147a624704aab66b737e036e9a9681f83c9223834a', text='The curious internet-dwelling user (maybe you)“What is machine learning?”', start_char_idx=None, end_char_idx=None, text_template='{metadata_

# CHROMA-DB

In [6]:
# create client and a new collection
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("test1")

### Set up ChromaVectorStore and load in data


In [8]:
# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)


In [9]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [10]:
service_context = ServiceContext.from_defaults(embed_model=embed_model)

******
Could not load OpenAI model. Using default LlamaCPP=llama2-13b-chat. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

******


llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/matansharon/Library/Caches/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q4_0     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_o

In [11]:

index = VectorStoreIndex.from_documents(documents,
                                        storage_context=storage_context,
                                        service_context=service_context)

In [14]:
# Query Data
query_engine = index.as_query_engine()

response = query_engine.query("what is the main keynote of ml?")

display(Markdown(f"{response}"))

Llama.generate: prefix-match hit

llama_print_timings:        load time = 20662.52 ms
llama_print_timings:      sample time =   125.41 ms /    49 runs   (    2.56 ms per token,   390.73 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time = 16221.14 ms /    49 runs   (  331.04 ms per token,     3.02 tokens per second)
llama_print_timings:       total time = 16724.18 ms


  Based on the provided context information, the main keynote of machine learning (ML) is to build a simple rule-based system that does not require ML, as stated in the quote from Google's Machine Learning Handbook.

### How to Persist: Saving to Disk

In [ ]:
db = chromadb.PersistentClient(path="./chroma_db")


chroma_collection = db.get_or_create_collection("DB_collection")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

service_context = ServiceContext.from_defaults(embed_model=embed_model,
                                               chunk_size=800,
                                               chunk_overlap=20)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, service_context=service_context
)

### Load from Disk

In [ ]:
# load from disk
db2 = chromadb.PersistentClient(path="./chroma_db")

chroma_collection = db2.get_or_create_collection("DB_collection")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(
    vector_store,
    service_context=service_context,
)

In [ ]:
# Query Data
query_engine = index.as_query_engine()

response = query_engine.query("What are some of the main contributions of this new Orca model?")

display(Markdown(f"{response}"))

# PINECONE

In [ ]:
!pip install -q pinecone-client

In [ ]:
import pinecone

In [ ]:
os.environ['PINECONE_API_KEY'] = "PINECONE_API_KEY"
os.environ['PINECONE_ENVIRONMENT'] = 'gcp-starter'


In [ ]:
# initialize connection to pinecone
pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'],
    environment=os.environ['PINECONE_ENVIRONMENT']
)

In [ ]:
# create the index if it does not exist already
index_name = 'llamaindex'
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=768,
        metric='cosine'
    )

In [ ]:
pinecone_index = pinecone.Index("llamaindex")
pinecone.describe_index("llamaindex")

In [ ]:
from llama_index.vector_stores import PineconeVectorStore

In [ ]:
# set add_sparse_vector=True to compute sparse vectors during upsert
from llama_index.storage.storage_context import StorageContext

In [ ]:
vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index,
    # add_sparse_vector=True,
)


In [ ]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(embed_model=embed_model)


In [ ]:
index = VectorStoreIndex.from_documents(documents,
                                        storage_context=storage_context,
                                        service_context=service_context
                                        )

In [ ]:
# Query Data
query_engine = index.as_query_engine()

response = query_engine.query("What is Big-Bench Hard test?")

display(Markdown(f"{response}"))